In [1]:
using LinearAlgebra
using LoopVectorization
using LinearMaps
using Arpack: Arpack
using ArnoldiMethod

function prepareDiag(L)
    diag = zeros(2^L)
    for state = 1:2^L
        for i = 1:L
            j = i==L ? 1 : i+1
            @inbounds diag[state] -= (((state >> (i-1))&1) == ((state >> (j-1))&1)) ? 1 : -1
        end
    end
    diag
end
    
function Hfunc!(C, B, diag, L)
    N = length(diag)
    @tturbo for state = 1:N
        C[state] = diag[state] * B[state]
    end
    for i = 1:L
        @tturbo for state = 1:N
            newstate = (state&(~(2^L))) ⊻ (1<<(i-1))
            c = newstate == 0
            newstate = !c*newstate + c*N # remove if statement
            C[newstate] -= B[state]
        end
    end
end
prepareHfunc!(diag, L) = (C, B) -> Hfunc!(C, B, diag, L)

struct MyLinMap{T, F, I<:Integer}
    f!::F; N::I; issymmetric::Bool; isposdef::Bool
end
MyLinMap(f!, N; issymmetric=false, isposdef=false) =
    MyLinMap{Float64, typeof(f!), typeof(N)}(f!, N, issymmetric, isposdef)
LinearAlgebra.mul!(y, A::MyLinMap, x) = A.f!(y, x)
Base.size(A::MyLinMap) = (A.N, A.N)
Base.size(A::MyLinMap, i::Integer) = size(A::MyLinMap)[i]
Base.eltype(A::MyLinMap{T}) where T = T
LinearAlgebra.issymmetric(A::MyLinMap) = A.issymmetric
LinearAlgebra.isposdef(A::MyLinMap) = A.isposdef

L = 20
diag_ = prepareDiag(L)
H! = prepareHfunc!(diag_, L)

H_LinearMap = LinearMap(H!, 2^L, ismutating=true, issymmetric=true, isposdef=false)
H_MyLinMap = MyLinMap(H!, 2^L, issymmetric = true, isposdef = false)
sol_exact = -2sum(abs(sin((n-1/2) * pi/L)) for n in 1:L)
@show sol_exact;

sol_exact = -25.49098968636475


In [2]:
@time e1, v1 = Arpack.eigs(H_LinearMap, nev=1, which=:SR)
@time e1, v1 = Arpack.eigs(H_LinearMap, nev=1, which=:SR)
e1[1]

 18.989504 seconds (14.82 M allocations: 1011.981 MiB, 1.02% gc time)
 10.119445 seconds (935 allocations: 200.040 MiB, 0.38% gc time)


-25.490989686364685

In [3]:
@time e2, v2 = Arpack.eigs(H_MyLinMap, nev=1, which=:SR)
@time e2, v2 = Arpack.eigs(H_MyLinMap, nev=1, which=:SR)
e2[1]

 10.488049 seconds (442.35 k allocations: 224.373 MiB, 1.48% gc time, 2.70% compilation time)
  9.427201 seconds (880 allocations: 200.038 MiB, 0.18% gc time)


-25.490989686364742

In [4]:
@time d1, h1 = partialschur(H_LinearMap, nev=1, which=SR())
@time d1, h1 = partialschur(H_LinearMap, nev=1, which=SR())
d1.eigenvalues[1]

  6.829463 seconds (12.33 M allocations: 930.490 MiB, 2.78% gc time, 48.00% compilation time)
  3.538457 seconds (100 allocations: 328.017 MiB, 0.25% gc time)


-25.490989686364667 + 0.0im

In [5]:
@time d2, h2 = partialschur(H_MyLinMap, nev=1, which=SR())
@time d2, h2 = partialschur(H_MyLinMap, nev=1, which=SR())
d2.eigenvalues[1]

  3.834794 seconds (1.84 M allocations: 417.458 MiB, 1.75% gc time, 6.73% compilation time)
  3.612403 seconds (99 allocations: 328.017 MiB, 2.20% gc time)


-25.49098968636474 + 0.0im